In [ ]:
%%html
<marquee style='width: 100%; color: red;'><b><li style="font-size:75px;">BERT</li></b></marquee>

## Reading the data

In [ ]:
import pandas as pd
import numpy as np
import os
from termcolor import colored
import warnings
!pip install ktrain
import ktrain
import tensorflow as tf

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Go to the file created in Colab and specify your path
dir = '/content/drive/MyDrive/IMDB_Data'

In [ ]:
# Reading in proper format from our mentioned file
df_train = pd.read_pickle(dir + '/train_features.pkl')
df_test  = pd.read_pickle(dir + '/test_features.pkl')

## BERT Model

The model used here is a pre-trained model by using K-train wrapper

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = ktrain.text.texts_from_df(train_df=df_train,
                                                                   text_column = 'Text',
                                                                   label_columns = 'Label',
                                                                   val_df = df_test,
                                                                   maxlen = 512,
                                                                   preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = ktrain.text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 512
done.


In [ ]:
# Initialise the model
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6)

In [ ]:
# Let's train the model
%%time
with tf.device('/gpu:0' and '/gpu:1'):
    learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 5336s 1s/step - loss: 0.3423 - accuracy: 0.8461 - val_loss: 0.2407 - val_accuracy: 0.9010
CPU times: user 42min 43s, sys: 7min 31s, total: 50min 15s
Wall time: 1h 29min 16s


## Analysis 

- BERT stands for Bidirectional Encoder Representations from Transformers. It is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context

- Due to being bilingual it can generalises well and when trained on huge corpus it generates good results

- This pre-training step is half the magic behind BERT’s success. This is because as we train a model on a large text corpus, our model starts to pick up the deeper and intimate understandings of how the language works


In [ ]:
learner.view_top_losses(n=1, preproc=preproc)